# 🌈 Quickstart

This page shows how to load a time-series spectroscopic dataset, do some basic calculations with it, generate some visualizations, and then save it out to another format. 

In [ ]:
from chromatic import read_rainbow, version
from astroquery.mast import Observations
import astropy.units as u

In [ ]:
version()

## 💾 Download Data
Let's download the [JWST Early Release Observation of HAT-P-18b](https://www.stsci.edu/jwst/science-execution/approved-programs/webb-first-image-observations), one of first exoplanet transit datasets to be gathered by the telescope. We'll get the default pipeline `x1dints` (Stage 3) outputs; there are lots of reasons why we shouldn't use these particular pipeline files for science, but they're useful for a quick initial look.

In [ ]:
Observations.download_file(
    f"mast:JWST/product/jw02734-o001_t001_niriss_clear-gr700xd-substrip256_x1dints.fits"
);

## 🧑‍💻 Load Data
Next, let's load that transit dataset into a `Rainbow` (🌈) object. These `chromatic` 🌈 objects keep track of how the brightness of source changes across both wavelength and time.

In [ ]:
rainbow = read_rainbow(
    "jw02734-o001_t001_niriss_clear-gr700xd-substrip256_x1dints.fits"
)

The 🌈 object we just loaded provides easy access to the different dimensions we might want from the dataset, arrays like wavelength, time, flux, or uncertainty. If appropriate, quantities will have `astropy` [Units](https://docs.astropy.org/en/stable/units/index.html). 

In [ ]:
rainbow.wavelength

In [ ]:
rainbow.time

In [ ]:
rainbow.flux

In [ ]:
rainbow.uncertainty

## 🧮 Do Basic Calculations

The absolute flux doesn't matter that much for many transit analyses, so let's use `.normalize()` to normalize out the median spectrum of the star, converting the data to relative brightness within each wavelength. This 🌈 action returns another 🌈 object, just with the brightness normalized.

In [ ]:
normalized = rainbow.normalize()

The dataset is really large, so for making some simple visualization it might help to average over bins of wavelength and/or time. Let's use `.bin()` to bin onto a (logarithmically) uniform wavelength grid, returning the binned 🌈.

In [ ]:
binned = normalized.bin(R=200, dt=4 * u.minute)

The times in this dataset are measured relative to some arbitrary time in the distant past. To make them easier to interpret we can phase-fold the times so they're measured relative to the mid-transit time, when the planet is directly between the star and us, according to the planet's [orbital properties from the NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/overview/HAT-P-18). Let's use `.fold()` to change the times, returning a phase-folded 🌈.

In [ ]:
folded = binned.fold(period=5.5080232 * u.day, t0=2459033.31735 * u.day)

## 🎨 Visualize

We can visualize the dataset by making a map of the star's brightness across both wavelength and time, an image in which each the brightness along row corresponds to a transit light curve at that wavelength. Let's use `.imshow()` to create this map for the normalized, binned, folded 🌈.

In [ ]:
folded.imshow();

It might be nice to look closely at the light curves within a particular wavelength range. Let's use `.imshow_interact()` to interactively explore the 🌈. Click and drag on the panel on the left to select the wavelegnth range to display as a light curve on the right.

In [ ]:
folded.imshow_interact()

What a dataset! It looks like there's something a little odd happening at about 2 microns (probably [contamination from another star or spectrograph order](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-features-and-caveats) and a starspot crossing just after mid-transit, but otherwise it's a remarkably beautiful transit from a very impressive telescope!

## 🧶 String Actions Together
Because many of the actions possible with 🌈 objects return other 🌈 objects, it's possible to connect multiple steps into a single command, building up complicated analysis stories with relatively succinct code.

In [ ]:
(
    rainbow.normalize()
    .flag_outliers()
    .bin(R=10)
    .fold(period=5.5080232 * u.day, t0=2459033.31735 * u.day)
    .plot()
);

## 💾 Save Data

Let's convert these data into a different format by saving it as a new file, which we might send around to share with our colleagues or publish along with a paper. `chromatic` can read and save 🌈 datasets with a variety of formats, to try to ease collaboration across different pipelines and toolkits.

In [ ]:
rainbow.save("jwst-hatp18b.rainbow.npy")

## 📚 Learn More

That's it! This quick tutorial highlighted `chromatic`'s abilities to...
- load in time-series spectra or multiwavelength light curves from formats like `x1dints`
- access core data variables like `wavelength`, `time`, `flux`, `uncertainty`
- perform calculations like `.normalize`, `.bin`, `.fold` 
- visualize the data with `.imshow`, `.imshow_interact`, `.plot`

Hopefully, you're now curious to read through the [User Guide](basics) to learn more about options for reading 🌈s, doing actions with 🌈s, visualizing 🌈s in different ways, and more! You can also run the `.help()` method associated with any 🌈 object to get a quick summary of what other methods are available for it>

In [ ]:
rainbow.help()